# Guia Spark ML

Leer dataset ML

In [1]:
##
#https://jdvelasq.github.io/courses/notebooks/pyspark/2-04-pyspark-SparkSQL.html
#https://guru99.es/pyspark-tutorial/
## findspark permite usar pyspark (interfaz de Python a Spark),
## desde cualquier programa escrito en Python.
##
import findspark
findspark.init()
##
## A continuación se inicializan las variables obligatorias
## requeridas para trabajar con Spark desde Python:
##
##  SparkContext representa la conexión al cluster de Spark.
##  SparkConf representa la configuración particular de una aplicación
##     escrita en Spark.
##  SparkSession representa la conexión para trabajar con SQL.
##
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

sparkConf = SparkConf().setAppName("ml_spark")
sc = SparkContext(conf=sparkConf)
spark = SparkSession(sc)

In [2]:
##
## Row representa una fila en un RDD
##
from pyspark.sql import Row
##
## Crea un DataFrame a partir del archivo con
## formato CSV
##
df_ml = spark.read.load("D:\\CLASES\ELECTIVA 3 BigData\\Clase5_Ejercicos_Spark\\DATOS\\icfes_ml.csv",
                     format="csv",
                     sep="|",
                     inferSchema= True,
                     encoding="utf-8",
                     header="true")
df_ml.printSchema()

root
 |-- periodo: integer (nullable = true)
 |-- estu_consecutivo: string (nullable = true)
 |-- estu_edad: double (nullable = true)
 |-- estu_tipo_documento: string (nullable = true)
 |-- estu_pais_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_nacimiento_dia: double (nullable = true)
 |-- estu_nacimiento_mes: double (nullable = true)
 |-- estu_nacimiento_anno: double (nullable = true)
 |-- estu_cod_reside_mcpio: double (nullable = true)
 |-- estu_reside_mcpio: string (nullable = true)
 |-- estu_reside_depto: string (nullable = true)
 |-- estu_zona_reside: double (nullable = true)
 |-- estu_area_reside: double (nullable = true)
 |-- cole_valor_pension: double (nullable = true)
 |-- estu_trabaja: double (nullable = true)
 |-- fami_estrato_vivienda: double (nullable = true)
 |-- estu_ies_cod_deseada: double (nullable = true)
 |-- estu_ies_deseada_nombre: string (nullable = true)
 |-- estu_ies_cod_mpio_deseada: double (nullable = true)
 |-- estu_ie

# Agrupar datos por clases

In [3]:
df_ml.createOrReplaceTempView("icfes_ml")
spark.sql("select clase,count(1) from icfes_ml group by clase").show()

+-----+--------+
|clase|count(1)|
+-----+--------+
|    1|  172623|
|    0|  374020|
+-----+--------+



# Explorar valores faltantes 

In [4]:
from pyspark.sql.functions import isnan, when, count, col
Dict_Null = {col:df_ml.filter(df_ml[col].isNull()).count() for col in df_ml.columns}

In [5]:
Dict_Null

{'periodo': 0,
 'estu_consecutivo': 0,
 'estu_edad': 355,
 'estu_tipo_documento': 0,
 'estu_pais_reside': 0,
 'estu_genero': 0,
 'estu_nacimiento_dia': 3816,
 'estu_nacimiento_mes': 3816,
 'estu_nacimiento_anno': 3816,
 'estu_cod_reside_mcpio': 3287,
 'estu_reside_mcpio': 0,
 'estu_reside_depto': 0,
 'estu_zona_reside': 3586,
 'estu_area_reside': 472,
 'cole_valor_pension': 1202,
 'estu_trabaja': 8713,
 'fami_estrato_vivienda': 4873,
 'estu_ies_cod_deseada': 489708,
 'estu_ies_deseada_nombre': 0,
 'estu_ies_cod_mpio_deseada': 489708,
 'estu_ies_mpio_deseada': 0,
 'estu_ies_dept_deseada': 0,
 'estu_mpio_presentacion': 0,
 'estu_dept_presentacion': 0,
 'estu_exam_cod_mpio_presentacio': 0,
 'estu_veces_estado': 721,
 'estu_carrdeseada_tipo': 490252,
 'fami_educa_padre': 1432,
 'fami_educa_madre': 1456,
 'fami_ocup_padre': 1523,
 'fami_ocup_madre': 1523,
 'fami_nivel_sisben': 640,
 'fami_pisos_hogar': 1209,
 'fami_personas_hogar': 658,
 'fami_telefono_fijo': 638,
 'fami_celular': 558,
 'fa

In [6]:
complete_dict = dict()
for col_name in df_ml.dtypes:
    if col_name[1]=='string':
        complete_dict[col_name[0]]="DESCONOCIDO"
    else:
        complete_dict[col_name[0]]=0    
complete_dict      
#df_ml.select([count(when(isnan(c), c)).alias(c) for c in df_ml.columns]).show()

{'periodo': 0,
 'estu_consecutivo': 'DESCONOCIDO',
 'estu_edad': 0,
 'estu_tipo_documento': 'DESCONOCIDO',
 'estu_pais_reside': 'DESCONOCIDO',
 'estu_genero': 'DESCONOCIDO',
 'estu_nacimiento_dia': 0,
 'estu_nacimiento_mes': 0,
 'estu_nacimiento_anno': 0,
 'estu_cod_reside_mcpio': 0,
 'estu_reside_mcpio': 'DESCONOCIDO',
 'estu_reside_depto': 'DESCONOCIDO',
 'estu_zona_reside': 0,
 'estu_area_reside': 0,
 'cole_valor_pension': 0,
 'estu_trabaja': 0,
 'fami_estrato_vivienda': 0,
 'estu_ies_cod_deseada': 0,
 'estu_ies_deseada_nombre': 'DESCONOCIDO',
 'estu_ies_cod_mpio_deseada': 0,
 'estu_ies_mpio_deseada': 'DESCONOCIDO',
 'estu_ies_dept_deseada': 'DESCONOCIDO',
 'estu_mpio_presentacion': 'DESCONOCIDO',
 'estu_dept_presentacion': 'DESCONOCIDO',
 'estu_exam_cod_mpio_presentacio': 0,
 'estu_veces_estado': 0,
 'estu_carrdeseada_tipo': 0,
 'fami_educa_padre': 0,
 'fami_educa_madre': 0,
 'fami_ocup_padre': 0,
 'fami_ocup_madre': 0,
 'fami_nivel_sisben': 0,
 'fami_pisos_hogar': 0,
 'fami_person

In [7]:
df_ml_complete = df_ml.na.fill(complete_dict)

In [8]:
Dict_Null = {col:df_ml_complete.filter(df_ml_complete[col].isNull()).count() for col in df_ml_complete.columns}

In [9]:
Dict_Null

{'periodo': 0,
 'estu_consecutivo': 0,
 'estu_edad': 0,
 'estu_tipo_documento': 0,
 'estu_pais_reside': 0,
 'estu_genero': 0,
 'estu_nacimiento_dia': 0,
 'estu_nacimiento_mes': 0,
 'estu_nacimiento_anno': 0,
 'estu_cod_reside_mcpio': 0,
 'estu_reside_mcpio': 0,
 'estu_reside_depto': 0,
 'estu_zona_reside': 0,
 'estu_area_reside': 0,
 'cole_valor_pension': 0,
 'estu_trabaja': 0,
 'fami_estrato_vivienda': 0,
 'estu_ies_cod_deseada': 0,
 'estu_ies_deseada_nombre': 0,
 'estu_ies_cod_mpio_deseada': 0,
 'estu_ies_mpio_deseada': 0,
 'estu_ies_dept_deseada': 0,
 'estu_mpio_presentacion': 0,
 'estu_dept_presentacion': 0,
 'estu_exam_cod_mpio_presentacio': 0,
 'estu_veces_estado': 0,
 'estu_carrdeseada_tipo': 0,
 'fami_educa_padre': 0,
 'fami_educa_madre': 0,
 'fami_ocup_padre': 0,
 'fami_ocup_madre': 0,
 'fami_nivel_sisben': 0,
 'fami_pisos_hogar': 0,
 'fami_personas_hogar': 0,
 'fami_telefono_fijo': 0,
 'fami_celular': 0,
 'fami_internet': 0,
 'fami_servicio_television': 0,
 'fami_computador':

# SPARK SQL

In [10]:
df_ml_complete.createOrReplaceTempView("icfes_ml_complete")
spark.sql("select clase,count(1) from icfes_ml_complete group by clase").show()

+-----+--------+
|clase|count(1)|
+-----+--------+
|    1|  172623|
|    0|  374020|
+-----+--------+



Agrupar por clase y departamento presentación

In [11]:
spark.sql("select"+
          " clase,estu_dept_presentacion,count(1)"+
          " from icfes_ml_complete"+
          " group by clase,estu_dept_presentacion order by estu_dept_presentacion").show(36*2)

+-----+----------------------+--------+
|clase|estu_dept_presentacion|count(1)|
+-----+----------------------+--------+
|    1|              AMAZONAS|     117|
|    0|              AMAZONAS|     607|
|    0|             ANTIOQUIA|   54936|
|    1|             ANTIOQUIA|   22153|
|    0|                ARAUCA|    2278|
|    1|                ARAUCA|     830|
|    0|             ATL�NTICO|   19680|
|    1|             ATL�NTICO|    8951|
|    1|                BOGOT�|   40434|
|    0|                BOGOT�|   59518|
|    1|               BOL�VAR|    7416|
|    0|               BOL�VAR|   17632|
|    0|                BOYAC�|   12261|
|    1|                BOYAC�|    6699|
|    0|                CALDAS|    8983|
|    1|                CALDAS|    3479|
|    1|               CAQUET�|    1043|
|    0|               CAQUET�|    3147|
|    1|              CASANARE|    1388|
|    0|              CASANARE|    3461|
|    1|                 CAUCA|    2961|
|    0|                 CAUCA|    8784|


# Filtrar departamento de Nariño

In [12]:
df_ml_complete=df_ml_complete.select(df_ml_complete.columns).filter(df_ml_complete['estu_dept_presentacion'] == 'NARIÑO')

In [13]:
df_ml_complete.groupby("clase").count().show()

+-----+-----+
|clase|count|
+-----+-----+
|    1| 4144|
|    0|11003|
+-----+-----+



# Contar municipios 

In [14]:
df_ml_complete.createOrReplaceTempView("icfes_ml_complete")
spark.sql("select estu_mpio_presentacion,count(1) from icfes_ml_complete group by estu_mpio_presentacion order by count(1) desc").show()

+----------------------+--------+
|estu_mpio_presentacion|count(1)|
+----------------------+--------+
|                 PASTO|    6063|
|               IPIALES|    2118|
|                TUMACO|    1923|
|             TUQUERRES|     641|
|              LA UNION|     500|
|                CUMBAL|     402|
|             SAMANIEGO|     386|
|               SANDONA|     361|
|               LA CRUZ|     308|
|             BARBACOAS|     294|
|               BUESACO|     274|
|             TAMINANGO|     270|
|             SAN PABLO|     260|
|              PUPIALES|     215|
|             GUACHUCAL|     160|
|      ALBAN (SAN JOSE)|     159|
|              RICAURTE|     158|
|              EL TAMBO|     156|
|             EL CHARCO|     144|
|         OLAYA HERRERA|      80|
+----------------------+--------+
only showing top 20 rows



# Contar municipios + clase 

In [15]:
df_ml_complete.createOrReplaceTempView("icfes_ml_complete")
spark.sql("select estu_mpio_presentacion,clase,count(1) from icfes_ml_complete group by estu_mpio_presentacion,clase order by estu_mpio_presentacion desc").show(100)

+----------------------+-----+--------+
|estu_mpio_presentacion|clase|count(1)|
+----------------------+-----+--------+
|             TUQUERRES|    0|     455|
|             TUQUERRES|    1|     186|
|                TUMACO|    1|     331|
|                TUMACO|    0|    1592|
|                TANGUA|    0|      55|
|                TANGUA|    1|      18|
|             TAMINANGO|    0|     218|
|             TAMINANGO|    1|      52|
|               SANDONA|    1|     110|
|               SANDONA|    0|     251|
|             SAN PABLO|    1|      96|
|             SAN PABLO|    0|     164|
|             SAMANIEGO|    0|     301|
|             SAMANIEGO|    1|      85|
|              RICAURTE|    1|      36|
|              RICAURTE|    0|     122|
|              PUPIALES|    0|     157|
|              PUPIALES|    1|      58|
|                 PASTO|    1|    1989|
|                 PASTO|    0|    4074|
|         OLAYA HERRERA|    1|      10|
|         OLAYA HERRERA|    0|      70|


# Contar colegios Nariño 

In [16]:
#cole_cod_icfes
#cole_nombre_sede
spark.sql("select estu_mpio_presentacion,count(distinct cole_nombre_sede||''||cole_cod_icfes) as c from icfes_ml_complete group by estu_mpio_presentacion order by count(cole_nombre_sede)").show(1000)

+----------------------+---+
|estu_mpio_presentacion|  c|
+----------------------+---+
|               LA TOLA|  1|
|     FRANCISCO PIZARRO|  2|
|              MOSQUERA|  1|
|               CONSACA|  3|
|                TANGUA|  6|
|         OLAYA HERRERA|  3|
|             EL CHARCO|  4|
|              EL TAMBO|  5|
|              RICAURTE| 11|
|      ALBAN (SAN JOSE)|  5|
|             GUACHUCAL|  8|
|              PUPIALES| 12|
|             SAN PABLO|  8|
|             TAMINANGO|  9|
|               BUESACO| 13|
|             BARBACOAS|  6|
|               LA CRUZ| 13|
|               SANDONA| 11|
|             SAMANIEGO| 13|
|                CUMBAL| 13|
|              LA UNION| 16|
|             TUQUERRES| 20|
|                TUMACO| 40|
|               IPIALES| 71|
|                 PASTO|138|
+----------------------+---+



# Filtrar municipios

In [17]:
df_filter=df_ml_complete.select(df_ml_complete.columns).filter(df_ml_complete['estu_mpio_presentacion'] == 'IPIALES')
#df_filter=df_ml_complete.select(df_ml_complete.columns).filter(df_ml_complete['estu_mpio_presentacion'] == 'TUMACO')

# Selecionar variables 

In [18]:
df_filter.printSchema()

root
 |-- periodo: integer (nullable = false)
 |-- estu_consecutivo: string (nullable = false)
 |-- estu_edad: double (nullable = false)
 |-- estu_tipo_documento: string (nullable = false)
 |-- estu_pais_reside: string (nullable = false)
 |-- estu_genero: string (nullable = false)
 |-- estu_nacimiento_dia: double (nullable = false)
 |-- estu_nacimiento_mes: double (nullable = false)
 |-- estu_nacimiento_anno: double (nullable = false)
 |-- estu_cod_reside_mcpio: double (nullable = false)
 |-- estu_reside_mcpio: string (nullable = false)
 |-- estu_reside_depto: string (nullable = false)
 |-- estu_zona_reside: double (nullable = false)
 |-- estu_area_reside: double (nullable = false)
 |-- cole_valor_pension: double (nullable = false)
 |-- estu_trabaja: double (nullable = false)
 |-- fami_estrato_vivienda: double (nullable = false)
 |-- estu_ies_cod_deseada: double (nullable = false)
 |-- estu_ies_deseada_nombre: string (nullable = false)
 |-- estu_ies_cod_mpio_deseada: double (nullable =

In [44]:
df_filter.groupby("estu_genero").count().show()
df_filter.groupby("estu_zona_reside").count().show()
df_filter.groupby("estu_area_reside").count().show()
df_filter.groupby("cole_valor_pension").count().show()
df_filter.groupby("estu_trabaja").count().show()
df_filter.groupby("fami_estrato_vivienda").count().show()
df_filter.groupby("estu_ies_cod_deseada").count().show() #No selec
df_filter.groupby("estu_ies_deseada_nombre").count().show()
df_filter.groupby("estu_veces_estado").count().show()
df_filter.groupby("estu_carrdeseada_tipo").count().show()
df_filter.groupby("fami_educa_padre").count().show()
df_filter.groupby("fami_educa_madre").count().show()
df_filter.groupby("fami_ocup_padre").count().show()
df_filter.groupby("fami_ocup_madre").count().show()
df_filter.groupby("fami_nivel_sisben").count().show()
df_filter.groupby("fami_pisos_hogar").count().show()
df_filter.groupby("fami_personas_hogar").count().show()
df_filter.groupby("fami_telefono_fijo").count().show()
df_filter.groupby("fami_celular").count().show()
df_filter.groupby("fami_internet").count().show()
df_filter.groupby("fami_servicio_television").count().show()
df_filter.groupby("fami_computador").count().show()
df_filter.groupby("fami_lavadora").count().show()
df_filter.groupby("fami_nevera").count().show()
df_filter.groupby("fami_horno").count().show()
df_filter.groupby("fami_dvd").count().show()
df_filter.groupby("fami_microondas").count().show()
df_filter.groupby("fami_automovil").count().show()
df_filter.groupby("fami_ing_fmiliar_mensual").count().show()
df_filter.groupby("estu_anos_preescolar").count().show()
df_filter.groupby("estu_ano_matricula_primero").count().show() #no sel 
df_filter.groupby("estu_ano_termino_quinto").count().show() #no sel
df_filter.groupby("estu_ano_matricula_sexto").count().show() #no sel
df_filter.groupby("estu_reprobo_primero").count().show() 
df_filter.groupby("estu_reprobo_segundo").count().show() 
df_filter.groupby("estu_anos_colegio_actual").count().show() 
df_filter.groupby("estu_cuantos_cole_estudio").count().show() 
df_filter.groupby("estu_razon_retiro").count().show() 
df_filter.groupby("estu_total_alumnos_curso").count().show() 
df_filter.groupby("estu_prestigioinstitucion").count().show() 
df_filter.groupby("estu_por_ubicacion").count().show() 
df_filter.groupby("estu_por_unicaqueofrece").count().show() 
df_filter.groupby("estu_por_amigosestudiando").count().show() 
df_filter.groupby("estu_por_otrarazon").count().show() 
df_filter.groupby("estu_por_orientacionvocacional").count().show() 
df_filter.groupby("estu_por_tradicionfamiliar").count().show() 
df_filter.groupby("estu_por_influenciaalguien").count().show() 
df_filter.groupby("cole_nombre_sede").count().show() 
df_filter.groupby("cole_calendario").count().show() #(F,A)
df_filter.groupby("cole_genero").count().show() #('F,X')
df_filter.groupby("cole_naturaleza").count().show() #('O,N')
df_filter.groupby("cole_bilingue").count().show()
df_filter.groupby("cole_jornada").count().show()
df_filter.groupby("cole_caracter").count().show()

+-----------+-----+
|estu_genero|count|
+-----------+-----+
|          F| 1092|
|       None|   15|
|          M| 1011|
+-----------+-----+

+----------------+-----+
|estu_zona_reside|count|
+----------------+-----+
|             8.0|    1|
|             0.0|   36|
|             7.0|    1|
|             4.0|    6|
|             3.0|    2|
|            10.0| 2068|
|             6.0|    1|
|             5.0|    3|
+----------------+-----+

+----------------+-----+
|estu_area_reside|count|
+----------------+-----+
|             0.0|   24|
|             1.0| 1356|
|             2.0|  738|
+----------------+-----+

+------------------+-----+
|cole_valor_pension|count|
+------------------+-----+
|               8.0|  121|
|               0.0| 1744|
|              11.0|   70|
|              10.0|    7|
|               9.0|  172|
|              12.0|    4|
+------------------+-----+

+------------+-----+
|estu_trabaja|count|
+------------+-----+
|         0.0| 1863|
|         7.0|   85|
|     

+-----------------------+-----+
|estu_ano_termino_quinto|count|
+-----------------------+-----+
|                    0.0| 1884|
|                 1987.0|    1|
|                 2001.0|   12|
|                 1997.0|    1|
|                 1992.0|    1|
|                 1990.0|    1|
|                 1995.0|    2|
|                 1996.0|    1|
|                 1986.0|    1|
|                 1998.0|    2|
|                 1985.0|    1|
|                 2000.0|    4|
|                 1994.0|    1|
|                 1999.0|    2|
|                 2002.0|   14|
|                 2004.0|  109|
|                 1991.0|    1|
|                 2005.0|   32|
|                 2003.0|   48|
+-----------------------+-----+

+------------------------+-----+
|estu_ano_matricula_sexto|count|
+------------------------+-----+
|                  1988.0|    1|
|                     0.0| 1884|
|                  1987.0|    1|
|                  2001.0|    3|
|                  1997.0|    2|

In [39]:
df=df_filter.select("estu_edad","estu_genero","estu_nacimiento_dia",
                "estu_nacimiento_mes","estu_nacimiento_anno",
                "estu_zona_reside","estu_area_reside","cole_valor_pension",
                "estu_trabaja","fami_estrato_vivienda","estu_veces_estado",
                "fami_educa_padre","fami_educa_madre",
                "fami_ocup_padre","fami_ocup_madre",
                "fami_nivel_sisben","fami_pisos_hogar","fami_personas_hogar",
                 "fami_telefono_fijo","fami_celular","fami_internet",
                 "fami_servicio_television","fami_computador",
                 "fami_lavadora","fami_nevera","fami_horno","fami_dvd",
                 "fami_microondas","fami_automovil","fami_ing_fmiliar_mensual",
                 "estu_anos_preescolar","estu_reprobo_primero",
                 "estu_anos_colegio_actual","estu_cuantos_cole_estudio",
                 "estu_razon_retiro","estu_total_alumnos_curso",
                 "estu_prestigioinstitucion","estu_por_ubicacion",
                 "estu_por_unicaqueofrece","estu_por_amigosestudiando",
                 "estu_por_otrarazon","estu_por_orientacionvocacional",
                 "estu_por_tradicionfamiliar","estu_por_influenciaalguien",
                 "cole_nombre_sede","cole_calendario",
                 "cole_genero","cole_naturaleza",
                 "cole_bilingue","cole_jornada","cole_caracter",
                 "prom_puntaje","clase"
                )

# Variables Dummies 

In [32]:
df.createOrReplaceTempView("icfes_dm")

In [21]:
import pyspark.sql.functions as F

In [22]:
print("ncol ant:",len(df.columns))

ncol ant: 53


Variables

estu_genero,cole_calendario,cole_genero,cole_naturaleza,cole_jornada,cole_caracter

In [57]:
df_dm=spark.sql("select dm.*,"+
"case when estu_genero='F' then 1 else 0 end estu_genero_f,"
"case when dm.cole_calendario='F' then 0 else 1 end cole_calendario_a,"+
"case when dm.cole_genero='F' then 0 else 1 end cole_genero_x,"+
"case when dm.cole_naturaleza='N' then 0 else 1 end cole_naturaleza_o,"+
"case when dm.cole_jornada='MAÑANA' then 1 else 0 end cole_jornada_manana,"+
"case when dm.cole_jornada='TARDE' then 1 else 0 end cole_jornada_tarde,"+
"case when dm.cole_jornada='NOCHE' then 1 else 0 end cole_jornada_noche,"+
"case when dm.cole_caracter='TECNICO' then 1 else 0 end cole_caracter_tecnico,"
"case when dm.cole_caracter='ACADEMICO' then 1 else 0 end cole_caracter_acaemico,"+
"case when dm.cole_caracter='NORMALISTA' then 1 else 0 end cole_caracter_normalista"
" from icfes_dm dm")

columnas=['cole_calendario','cole_genero','cole_naturaleza',
          'cole_jornada','cole_caracter','estu_genero']
for columna in columnas:
    df_dm=df_dm.drop(columna)
df_dm.printSchema()

root
 |-- estu_edad: double (nullable = false)
 |-- estu_nacimiento_dia: double (nullable = false)
 |-- estu_nacimiento_mes: double (nullable = false)
 |-- estu_nacimiento_anno: double (nullable = false)
 |-- estu_zona_reside: double (nullable = false)
 |-- estu_area_reside: double (nullable = false)
 |-- cole_valor_pension: double (nullable = false)
 |-- estu_trabaja: double (nullable = false)
 |-- fami_estrato_vivienda: double (nullable = false)
 |-- estu_veces_estado: double (nullable = false)
 |-- fami_educa_padre: double (nullable = false)
 |-- fami_educa_madre: double (nullable = false)
 |-- fami_ocup_padre: double (nullable = false)
 |-- fami_ocup_madre: double (nullable = false)
 |-- fami_nivel_sisben: double (nullable = false)
 |-- fami_pisos_hogar: double (nullable = false)
 |-- fami_personas_hogar: double (nullable = false)
 |-- fami_telefono_fijo: double (nullable = false)
 |-- fami_celular: double (nullable = false)
 |-- fami_internet: double (nullable = false)
 |-- fami_s

# Actividad 

Realizar un consulta spark sql sobre el conjunto de datos df_dm que  cuente cuántos estudiantes hay por cada variable socioeconómica agrupados por colegio.

In [58]:
print("ncol ant:",len(df_dm.columns))

columna="cole_nombre_sede"
categ = df_dm.select(columna).distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col(columna) == cat,1).otherwise(0)\
            .alias(columna+"_"+str(cat).lower().replace(" ","_").replace(".","")) for cat in categ]

df_dm = df_dm.select(df_dm.columns+exprs)
print("ncol new:",len(df_dm.columns))
df_dm=df_dm.drop(columna)

ncol ant: 57
ncol new: 118


In [59]:
df_dm.printSchema()
#df.groupby("clase").count().show()

root
 |-- estu_edad: double (nullable = false)
 |-- estu_nacimiento_dia: double (nullable = false)
 |-- estu_nacimiento_mes: double (nullable = false)
 |-- estu_nacimiento_anno: double (nullable = false)
 |-- estu_zona_reside: double (nullable = false)
 |-- estu_area_reside: double (nullable = false)
 |-- cole_valor_pension: double (nullable = false)
 |-- estu_trabaja: double (nullable = false)
 |-- fami_estrato_vivienda: double (nullable = false)
 |-- estu_veces_estado: double (nullable = false)
 |-- fami_educa_padre: double (nullable = false)
 |-- fami_educa_madre: double (nullable = false)
 |-- fami_ocup_padre: double (nullable = false)
 |-- fami_ocup_madre: double (nullable = false)
 |-- fami_nivel_sisben: double (nullable = false)
 |-- fami_pisos_hogar: double (nullable = false)
 |-- fami_personas_hogar: double (nullable = false)
 |-- fami_telefono_fijo: double (nullable = false)
 |-- fami_celular: double (nullable = false)
 |-- fami_internet: double (nullable = false)
 |-- fami_s

In [60]:
df_dm.repartition(1).write.csv('D:\\CLASES\ELECTIVA 3 BigData\\Clase5_Ejercicos_Spark\\DATOS\\data_icfes_ml.csv',sep='|',header="true")